# Create Dataset from Raw Data

Please note, the original data and cell output in this workbook is not shared due to privacy. The original excel workbook is messy: full of typos, wrong data types, two data points in a cell. 

This workbook cleans the data up before creating a new data set. 

In [ ]:
# libraries

import pandas as pd
import datetime as dt
import re

In [ ]:
# load data and check

df_first_tab = pd.read_excel("../Data/raw_data.xlsx", sheet_name=0, usecols=["Date","Time","Bottle (ml)","Poo","Wee","Other"], nrows=237)

df_first_tab.head()

In [ ]:
# load data and check - data exists on a second worksheet as well

df_second_tab = pd.read_excel("../Data/raw_data.xlsx", sheet_name=1, parse_dates=False)

df_second_tab.head()

In [ ]:
# combine both worksheets to one dataframe

df = pd.concat([df_first_tab, df_second_tab], ignore_index=True)

df.head()

In [ ]:
# check data types

df.dtypes

In [ ]:
## Create Date Column

# regex to find dates
p = re.compile(r"[\d]{1,2}/[\d]{1,2}/[\d]{2,4}")

# function to extract dates and deal with NaT entries

def date_fixer(column: pd.Series) -> list:
    returned_dates = []
    for date in column:
        if isinstance(date, dt.datetime):
            if pd.isnull(date) is False:
                converted_date = pd.to_datetime(date)
                returned_dates.append(converted_date)
                last_recorded_date = converted_date
            else: # pd.isnull(date) is True
                returned_dates.append(last_recorded_date)
        elif pd.isnull(date) is True:
            returned_dates.append(last_recorded_date)
        elif isinstance(date, str):
            m = p.search(date)
            if m:
                converted_date = pd.to_datetime(m.group(), dayfirst=True)
                returned_dates.append(converted_date)
                last_recorded_date = converted_date
            else:
                returned_dates.append(last_recorded_date)
        else:
            returned_dates.append(last_recorded_date)
    return returned_dates

proper_dates = date_fixer(df["Date"])

proper_dates = pd.Series(proper_dates)

print(proper_dates.shape)
print(proper_dates.info)

In [ ]:
# One time has been read as a time. Converting it to string to avoid changing function below
# problem time
print(df.iloc[43,1])
# fixing issue
df.iloc[43,1]="*0000"
# print corrected data
print(df.iloc[43,1])

In [ ]:
## Time

# function to add time data to new_date and set timezone to Australia

def add_time(time: str, date: pd.Timestamp) -> pd.Timestamp:
    # regex to find times 
    p = re.compile(r"[\d]{1,2}[\d]{1,2}")
    match = p.search(time)
    match_text = match.group()
    # find matches and get hour and minute data
    if len(match_text) == 3:
        hour = int(match_text[0])
        minute = int(match_text[1:3])
    elif len(match_text) == 4:
        hour = int(match_text[0:2])
        minute = int(match_text[2:4])
    else:
        raise ValueError("Did not find match")
    # get date and replace with new time
    date_time = date.replace(hour=hour, minute=minute)
    return date_time

time_strings = df["Time"].astype(str)

dates_times = pd.Series(map(add_time, time_strings, proper_dates))

dates_times


In [ ]:
# Feed

def feed(recorded_data) -> str:
    if pd.isna(recorded_data):
        return None
    else:
        return "Feed"

Feeds = pd.Series(map(bottle_feed, df["Feed"]))

Feeds_dummies = pd.get_dummies(Bottles)

Feeds_dummies

In [ ]:
## Poo column

# Check data
# If NA print how many instances
print(f"Numer of NA values: {df[df["Poo"].isna()].shape[0]}")

# Get all unique values
print(f"Unique values are: {[val for val in df["Poo"].unique()]}")

In [ ]:
# fix NA values and normalise
df["Poo"].fillna("No", inplace=True)

# create normalised Series using match statement and map
def poo_fixer(poo: str) -> str:
    match poo:
        case "Yes":
            return "Poo"
        case "Yes ":
            return "Poo"
        case "yes":
            return "Poo"
        case "Tiny amount":
            return "Poo"
        case "No":
            return None
        case "No ":
            return None
        case "Small":
            return "Poo"
        case " No":
            return None
        case "no ":
            return None
        case " ":
            return None

fixed_poos = pd.Series(map(poo_fixer, df["Poo"]))

fixed_poos = pd.get_dummies(new_poos)

fixed_poos

In [ ]:
## Wee Column

# Check data
# If NA print how many instances
print(f"Numer of NA values: {df[df["Wee"].isna()].shape[0]}")

# Get all unique values
print(f"Unique values are: {[val for val in df["Wee"].unique()]}")

In [ ]:
# fix NA values and normalise
df["Wee"].fillna("No", inplace=True)

# create normalised Series using match statement and map
def wee_fixer(wee: str) -> str:
    match wee:
        case "Yes":
            return "Wee"
        case "Yes ":
            return "Wee"
        case "Yea":
            return "Wee"
        case "yes":
            return "Wee"
        case "Yrs":
            return "Wee"
        case "No":
            return None

fixed_wees = pd.Series(map(wee_fixer, df["Wee"]))

fixed_wees = pd.get_dummies(new_wees)

fixed_wees

In [ ]:
# Get Vomits

# Use regex to find 'vomit' matches
def vomit_finder(note: str) -> str:
    if isinstance(note, str):
        p = re.compile(r"vomit", flags=re.I)
        m = p.search(note)
        if m:
            return "vomited"
        else:
            return None
    else:
        None

vomits = pd.Series(map(vomit_finder, df["Other"]))

vomits = pd.get_dummies(vomits)

vomits

In [ ]:
## Stitch dataframe together

data = {"DateTime": dates_times,
        "Feed": Bottle_dummies["Bottle"],
        "Poo": new_poos["Poo"],
        "Wee": new_wees["Wee"],
        "Vomit": vomits["vomited"]}

df_clean_data = pd.DataFrame(data=data)

df_clean_data.set_index("DateTime", inplace=True)

df_clean_data

In [ ]:
df_clean_data.info()

In [ ]:
df_clean_data.describe()

In [ ]:
# save new cleanded dataset
df_clean_data.to_csv("../Data/dataLeon.csv")